In [1]:
import pandas as pd
import numpy as np
import cvxopt
from cvxopt import matrix
from cvxopt import solvers
from tqdm import tqdm
from libsvm import svmutil
import time
import csv
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from libsvm import svmutil
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [2]:
#paramters

p1 = "fashion_mnist/train.csv" #training file
p2 = "fashion_mnist/test.csv"  #testing file
p3 = "fashion_mnist/val.csv"  #validation file

In [3]:
def read_data(p1):
    df_train_data = pd.read_csv(p1,encoding='latin-1', header=None)
    XY_train_78 = df_train_data[(df_train_data[784]==7) | (df_train_data[784]==8)]
    XY_train_78 = XY_train_78.reset_index(drop=True)
    XY_train_78.loc[XY_train_78[784] == 7,784] = -1
    XY_train_78.loc[XY_train_78[784] == 8,784] = 1
    XY_train = np.array(XY_train_78)
    X_train = XY_train[:,0:784]
    Y_train = XY_train[:,784:785]
    X_train_scaled = X_train/255
    
    return X_train_scaled,Y_train

In [4]:
def read_data_nofilter(p1):
    df_train_data = np.array(pd.read_csv(p1,encoding='latin-1', header=None))
    X_train = df_train_data[:,0:784]
    Y_train = df_train_data[:,784:785]
    X_train_scaled = X_train/255
    
    return X_train_scaled,Y_train

In [5]:
x1,y1 = read_data(p1)
m,n= x1.shape
print('shape of x1 :',x1.shape)
print('shape of y1 :',y1.shape)

shape of x1 : (4500, 784)
shape of y1 : (4500, 1)


In [6]:
def cal_k(x1):
    K = np.zeros((m,m))
    for i in tqdm(range(m)):
        for j in range(m):
            temp1 = np.linalg.norm(x1[i] - x1[j])
            temp2 = temp1**2
#             temp2 = temp1
            temp2 = 0.05*temp2
            temp2 = - temp2
            temp3 = np.exp(temp2)
            K[i][j] = temp3
    return K

In [33]:
def calc_params(x1,y1):
    C = 1
    y_mat = np.dot(y1,y1.T)
    K = cal_k(x1)
    print('shape of K',K.shape)
    
    H = y_mat*K*1.
    
    P = matrix(H)
    q = matrix(-np.ones((m, 1)))
    t1 = np.eye(m)
    t2 = (-1)*np.eye(m)
    G = matrix(np.vstack((t2,t1)))
    m_zeros = np.zeros(m)
    m_ones = np.ones(m)
    m_onesc = m_ones*C
    h_npstack = np.hstack((m_zeros, m_onesc))
    h = matrix(h_npstack)
    A = matrix(y1.reshape(1, -1))
    b = matrix(np.zeros(1))
    
    return P, q, G, h, A, b

In [34]:
def cal_alphas():
    P, q, G, h, A, b = calc_params(x1,y1)
    sol = cvxopt.solvers.qp(P, q, G, h, A, b) 
    return np.array(sol['x'])

In [9]:
t0 = time.time()
alphas = cal_alphas()
print('Time to calculare alphas in seconds = ',time.time()-t0)

100%|██████████| 4500/4500 [03:28<00:00, 21.62it/s]


shape of K (4500, 4500)
     pcost       dcost       gap    pres   dres
 0: -1.4021e+02 -7.5807e+03  4e+04  2e+00  2e-15
 1: -7.7474e+01 -3.9413e+03  7e+03  3e-01  2e-15
 2: -3.7302e+01 -1.0367e+03  1e+03  4e-02  5e-15
 3: -7.4177e+01 -2.7628e+02  2e+02  6e-03  3e-15
 4: -9.3886e+01 -1.5833e+02  7e+01  1e-03  2e-15
 5: -1.0307e+02 -1.2356e+02  2e+01  3e-04  1e-15
 6: -1.0675e+02 -1.1333e+02  7e+00  5e-05  1e-15
 7: -1.0821e+02 -1.0986e+02  2e+00  6e-06  1e-15
 8: -1.0865e+02 -1.0897e+02  3e-01  7e-07  1e-15
 9: -1.0875e+02 -1.0879e+02  4e-02  6e-08  1e-15
10: -1.0876e+02 -1.0876e+02  1e-03  1e-09  2e-15
11: -1.0876e+02 -1.0876e+02  2e-05  2e-11  2e-15
Optimal solution found.
Time to calculare alphas in seconds =  229.50530457496643


In [10]:
def calc_nonzero_alphas(alphas_param):
    nonzero_alphas_list = []
    support_vectors = 0
    for i in alphas_param:
        if i > 1e-5 :
            nonzero_alphas_list.append(i)
            support_vectors +=1
        else :
            nonzero_alphas_list.append(0)
    print('No. of Support vectors :',support_vectors)
    return np.array(nonzero_alphas_list)

In [11]:
alphas_reshape = alphas.reshape(m,)

In [12]:
#updating alpha values with 0 where alpha_i is < 1e-5
#It prints the number of support vectors
alpha_vlaues = calc_nonzero_alphas(alphas_reshape)

No. of Support vectors : 1063


In [13]:
def cal_b(alpha_vlaues):
    for i in tqdm(range(m)):
        if alpha_vlaues[i]!=0:
            sum_val = 0
            for j in range(m):      
                t1 = alphas[j][0]*y1[j][0]
                t2 = np.linalg.norm(x1[i]-x1[j])
                t2 = t2**2
                t3 = -0.05*t2
                t4 = np.exp(t3)
                t5  =t1*t4
                sum_val = sum_val + t5
            alpha_vlaues[i] = y1[i][0] - sum_val

    return sum(alpha_vlaues)/np.count_nonzero(alpha_vlaues)


In [14]:
t0 = time.time()
finalb = cal_b(alpha_vlaues)
print('\n\nTraining time in seconds = ',time.time()-t0)

100%|██████████| 4500/4500 [00:51<00:00, 87.28it/s] 



Training time in seconds =  51.565696001052856


In [15]:
def cal_accuracy(xtest,ytest):
    pred_list = []
    crct_cases=0
    n = len(xtest)
    alphasy1 = alphas.flatten()*y1.flatten()
    for i in tqdm(range(n)):
        a1 = xtest[i]
        Kt = np.zeros(m)
        for i in range(m):
            t1 = np.linalg.norm(x1[i]-a1)
            t2 = t1**2
            Kt[i] = np.exp(-0.05*t2)
        
        pred_val = np.matmul(alphasy1,Kt.T)+finalb
        if pred_val < 0:
            pred_list.append(-1)

        if pred_val > 0:
            pred_list.append(1)
    
    for i in tqdm(range(n)):
        if ytest[i] == pred_list[i]:
            crct_cases +=1
    print('correct cases :',crct_cases)
    print('total test examples :',n)
    return crct_cases/n

In [16]:
xtest,ytest = read_data(p2)
print('Accuracy on test data :',cal_accuracy(xtest,ytest)*100,'%')

100%|██████████| 1000/1000 [00:00<00:00, 480722.52it/s]

correct cases : 995
total test examples : 1000
Accuracy on test data : 99.5 %


In [17]:
xval,yval = read_data(p3)
print('Accuracy on validation data :',cal_accuracy(xval,yval)*100,'%')

100%|██████████| 500/500 [00:00<00:00, 572523.07it/s]

correct cases : 493
total test examples : 500
Accuracy on validation data : 98.6 %


In [18]:
xtrain,ytrain = read_data(p1)
print('Accuracy on test data :',cal_accuracy(xtrain,ytrain)*100,'%')

100%|██████████| 4500/4500 [00:00<00:00, 796252.45it/s]

correct cases : 4500
total test examples : 4500
Accuracy on test data : 100.0 %


#Part_B_Q1.C

In [19]:
xtrain,ytrain = read_data(p1)
ytrain = ytrain.reshape(m,)
linear_model = SVC(kernel='linear')

In [20]:
%%timeit
linear_model.fit(xtrain, ytrain)

628 ms ± 43.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
xtest,ytest = read_data(p2)
xval,yval = read_data(p3)

In [22]:
print('Mean of W of the linear SVM Model :',np.mean(linear_model.coef_))
print('b value of the linear SVM Model :',linear_model.intercept_[0])
print('No. of support vectors of the linear SVM Model :',linear_model.n_support_[0] + linear_model.n_support_[1])
print('Accuracy of Test data on the linear SVM Model :',linear_model.score(xtest,ytest)*100,'%')
print('Accuracy of Validation data on the linear SVM Model :',linear_model.score(xval,yval)*100,'%')

Mean of W of the linear SVM Model : 0.019834433262008663
b value of the linear SVM Model : -0.9348381492157227
No. of support vectors of the linear SVM Model : 120
Accuracy of Test data on the linear SVM Model : 99.4 %
Accuracy of Validation data on the linear SVM Model : 99.2 %


In [23]:
Gaussian_model = SVC(kernel='rbf',gamma=0.05)

In [24]:
%%timeit
Gaussian_model.fit(xtrain, ytrain)

3.91 s ± 50.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
# print('Mean of W of the gaussian SVM Model :',np.mean(Gaussian_model.coef_))
print('b value of the gaussian SVM Model :',Gaussian_model.intercept_[0])
print('No. of support vectors of the gaussian SVM Model :',Gaussian_model.n_support_[0] + linear_model.n_support_[1])
print('Accuracy of Test data on the gaussian SVM Model :',Gaussian_model.score(xtest,ytest)*100,'%')
print('Accuracy of Validation data on the gaussian SVM Model :',Gaussian_model.score(xval,yval)*100,'%')

b value of the gaussian SVM Model : 0.7056494588279296
No. of support vectors of the gaussian SVM Model : 338
Accuracy of Test data on the gaussian SVM Model : 99.5 %
Accuracy of Validation data on the gaussian SVM Model : 98.6 %


In [26]:
#Que2.b
#Multi Class SVM
x_train_nf,y_train_nf = read_data_nofilter(p1)
xtest_nf,ytest_nf = read_data_nofilter(p2)
xval_nf,yval_nf = read_data_nofilter(p3)

In [27]:
print('Que_2(b)Learning with Gaussian Kernel')
multi_model = SVC(kernel='rbf',gamma=0.05)
t0 = time.time()
multi_model.fit(x_train_nf, y_train_nf)
print('Training time in seconds = ',time.time()-t0)

print('Accuracy on test data :',multi_model.score(xtest_nf, ytest_nf))
print('Accuracy on validation data :',multi_model.score(xval_nf, yval_nf))

Que_2(b)Learning with Gaussian Kernel


/home/manu/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training time in seconds =  217.7474546432495
Accuracy on test data : 0.8808
Accuracy on validation data : 0.8792


In [30]:
ytest_predicted = multi_model.predict(xtest_nf) 
yval_predicted = multi_model.predict(xval_nf) 

In [31]:
ConfusionMatrix2 = confusion_matrix(yval_nf,yval_predicted)
print('\n\nConfusion matrix for test data(part a)\n\n')
print(ConfusionMatrix2)
ConfusionMatrix2 = confusion_matrix(yval_nf,yval_predicted)

ConfusionMatrix1 = confusion_matrix(ytest_nf,ytest_predicted)
print('\n\nConfusion matrix for test data\n\n')
print(ConfusionMatrix1)



Confusion matrix for test data(part a


[[212   0   1   8   0   0  26   0   3   0]
 [  0 237   3   7   0   0   2   0   1   0]
 [  5   0 206   3  18   0  13   0   5   0]
 [  6   0   0 228   6   0   9   0   1   0]
 [  1   1  24   8 200   0  15   0   1   0]
 [  0   0   0   1   0 241   0   2   1   5]
 [ 34   0  28   3  19   0 165   0   1   0]
 [  0   0   0   0   0   8   0 230   1  11]
 [  0   0   1   1   1   0   1   2 244   0]
 [  0   0   0   0   0   6   0   8   1 235]]


Confusion matrix for test data


[[433   0   5  11   3   0  38   0  10   0]
 [  1 482   4   9   0   0   4   0   0   0]
 [  5   0 411   7  37   0  32   0   8   0]
 [ 12   0   3 457   9   0  14   0   5   0]
 [  3   1  41  13 399   0  38   0   5   0]
 [  0   0   0   0   0 473   0  16   5   6]
 [ 80   0  55   9  34   0 315   0   7   0]
 [  0   0   0   0   0  14   0 471   1  14]
 [  1   0   1   1   2   2   2   2 489   0]
 [  0   0   0   0   0  11   0  14   1 474]]
